In [33]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# Read the data from the CSV file
data = pd.read_csv("Students_Grading_Dataset.csv")
data.head()

,Student_ID,First_Name,Last_Name,Email,Gender,Age,Department,Attendance (%),Midterm_Score,Final_Score,...,Projects_Score,Total_Score,Grade,Study_Hours_per_Week,Extracurricular_Activities,Internet_Access_at_Home,Parent_Education_Level,Family_Income_Level,Stress_Level (1-10),Sleep_Hours_per_Night
0,S1000,Omar,Williams,student0@university.com,Female,22,Engineering,52.29,55.03,57.82,...,85.90,56.09,F,6.2,No,Yes,High School,Medium,5,4.7
1,S1001,Maria,Brown,student1@university.com,Male,18,Engineering,97.27,97.23,45.80,...,55.65,50.64,A,19.0,No,Yes,None,Medium,4,9.0
2,S1002,Ahmed,Jones,student2@university.com,Male,24,Business,57.19,67.05,93.68,...,73.79,70.30,D,20.7,No,Yes,Master's,Low,6,6.2
3,S1003,Omar,Williams,student3@university.com,Female,24,Mathematics,95.15,47.79,80.63,...,92.12,61.63,A,24.8,Yes,Yes,High School,High,3,6.7
4,S1004,John,Smith,student4@university.com,Female,23,CS,54.18,46.59,78.89,...,68.42,66.13,F,15.4,Yes,Yes,High School,High,2,7.1


In [34]:
# Drop uneccessary columns
data = data.drop(columns = ["Student_ID", "First_Name", "Last_Name", "Email"])

In [35]:
# Check for missing values
print(data.isnull().sum())

# Fill missing values with the mean of the column for same stress level value
data["Attendance (%)"] = data.groupby("Stress_Level (1-10)")["Attendance (%)"].transform(lambda x: x.fillna(x.mean()))
data["Assignments_Avg"] = data.groupby("Stress_Level (1-10)")["Assignments_Avg"].transform(lambda x: x.fillna(x.mean()))

# Fill missing Parent_Education_Level with a new category "Unknown"
data["Parent_Education_Level"] = data["Parent_Education_Level"].fillna("Unknown")

# Perform one-hot encoding for the Gender feature
gender_encoded = pd.get_dummies(data['Gender'], prefix='Gender')
data = pd.concat([data, gender_encoded], axis=1)

# Drop the original Gender column and one of the encoded columns to reduce collinearity
data = data.drop(columns=['Gender', 'Gender_Female'])

# Perform one-hot encoding for the Department feature
department_encoded = pd.get_dummies(data['Department'], prefix='Department')
data = pd.concat([data, department_encoded], axis=1)

# Drop the original Department column
data = data.drop(columns=['Department'])

# Perform ordinal encoding for the Grade feature (F as 0, A as 4)
ordinal_encoder = OrdinalEncoder(categories=[['F', 'D', 'C', 'B', 'A']])
data['Grade'] = ordinal_encoder.fit_transform(data[['Grade']])
data['Grade'] = data['Grade'].astype(int)

# Perform ordinal encoding for the Parent_Education_Level feature (None as 0, High School as 1, Unknown as 2, Bachelor's as 3, Master's as 4, PhD as 5)
ordinal_encoder_parent = OrdinalEncoder(categories=[['None', 'High School', 'Unknown', "Bachelor's", "Master's", 'PhD']])
data['Parent_Education_Level'] = ordinal_encoder_parent.fit_transform(data[['Parent_Education_Level']])
data['Parent_Education_Level'] = data['Parent_Education_Level'].astype(int)

# Perform ordinal encoding for the Family_Income_Level feature (low as 0, medium as 1, high as 2)
ordinal_encoder_income = OrdinalEncoder(categories=[['Low', 'Medium', 'High']])
data['Family_Income_Level'] = ordinal_encoder_income.fit_transform(data[['Family_Income_Level']])
data['Family_Income_Level'] = data['Family_Income_Level'].astype(int)

# Convert Extracurricular_Activities and Internet_Access_at_Home to boolean
data['Extracurricular_Activities'] = data['Extracurricular_Activities'].map({'Yes': True, 'No': False})
data['Internet_Access_at_Home'] = data['Internet_Access_at_Home'].map({'Yes': True, 'No': False})

Gender                          0
Age                             0
Department                      0
Attendance (%)                516
Midterm_Score                   0
Final_Score                     0
Assignments_Avg               517
Quizzes_Avg                     0
Participation_Score             0
Projects_Score                  0
Total_Score                     0
Grade                           0
Study_Hours_per_Week            0
Extracurricular_Activities      0
Internet_Access_at_Home         0
Parent_Education_Level        984
Family_Income_Level             0
Stress_Level (1-10)             0
Sleep_Hours_per_Night           0
dtype: int64


In [37]:
# Check for missing values
#print(data.isnull().sum())
data.head()

,Age,Attendance (%),Midterm_Score,Final_Score,Assignments_Avg,Quizzes_Avg,Participation_Score,Projects_Score,Total_Score,Grade,...,Internet_Access_at_Home,Parent_Education_Level,Family_Income_Level,Stress_Level (1-10),Sleep_Hours_per_Night,Gender_Male,Department_Business,Department_CS,Department_Engineering,Department_Mathematics
0,22,52.29,55.03,57.82,84.220000,74.06,3.99,85.90,56.09,0,...,True,1,1,5,4.7,0,0,0,1,0
1,18,97.27,97.23,45.80,74.121996,94.24,8.32,55.65,50.64,4,...,True,0,1,4,9.0,1,0,0,1,0
2,24,57.19,67.05,93.68,67.700000,85.70,5.05,73.79,70.30,1,...,True,4,0,6,6.2,1,1,0,0,0
3,24,95.15,47.79,80.63,66.060000,93.51,6.54,92.12,61.63,4,...,True,1,2,3,6.7,0,0,0,0,1
4,23,54.18,46.59,78.89,96.850000,83.70,5.97,68.42,66.13,0,...,True,1,2,2,7.1,0,0,1,0,0
